In [51]:
# data preproccessing
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# Load the data
data = pd.read_csv('C:\data_set\depression.csv')



# Remove two columns
data = data[["Gender","Age","Education Level","Marital Status","Household Size","Household Income","Full Time Work","Work Type","Time In Current Job","Out Of Work","Trouble Sleeping History","Sleep Hours","Vigorous Recreation","Sedentary Time","Vigorous Work","Moderate Work","Cant Work","Lifetime Alcohol Consumption","Pregnant","Health Problem Heart","Health Problem Diabetes","Current Smoker","Depression"]]
# Replace "missing" with NaN in the entire dataframe
data = data.apply(lambda x: x.str.strip().replace('Missing', pd.NA) if x.dtype == "object" else x)
encoder = LabelEncoder()
data['Gender'] = encoder.fit_transform(data['Gender'])
data["Depression"] = data["Depression"].replace({"Not Depressed": 0, "Depressed": 1})
# Reverse the label encoding
data.head(2)

,Gender,Age,Education Level,Marital Status,Household Size,Household Income,Full Time Work,Work Type,Time In Current Job,Out Of Work,...,Sedentary Time,Vigorous Work,Moderate Work,Cant Work,Lifetime Alcohol Consumption,Pregnant,Health Problem Heart,Health Problem Diabetes,Current Smoker,Depression
0,0,44,Some College or AA Degree,Married,4,$75K+,<NA>,Private Wage Worker,324.0,<NA>,...,150.0,No,No,No,Yes,No,No,No,<NA>,0
1,1,70,College Graduate or Above,Married,2,$75K+,No,Private Wage Worker,96.0,<NA>,...,150.0,No,No,No,<NA>,<NA>,No,No,<NA>,0


In [52]:
data.shape                                                          

(36259, 23)

In [53]:
data.isnull().sum()

Gender                              0
Age                                 0
Education Level                  2157
Marital Status                   1636
Household Size                      0
Household Income                 1813
Full Time Work                  30492
Work Type                       16284
Time In Current Job                 0
Out Of Work                     21461
Trouble Sleeping History            0
Sleep Hours                         0
Vigorous Recreation                 3
Sedentary Time                      0
Vigorous Work                       0
Moderate Work                       0
Cant Work                           0
Lifetime Alcohol Consumption    22323
Pregnant                        28366
Health Problem Heart                0
Health Problem Diabetes             0
Current Smoker                  20789
Depression                          0
dtype: int64

In [54]:
# Drop the columns that have more than 10000 null values
data = data.dropna(axis=1, thresh=len(data)-10000)

# Drop the rows that contain any null value
data = data.dropna(axis=0)

In [55]:
data.corr()["Depression"]

Gender                -0.077534
Age                   -0.010262
Household Size        -0.007909
Time In Current Job   -0.087375
Sleep Hours           -0.085884
Sedentary Time         0.033815
Depression             1.000000
Name: Depression, dtype: float64

In [56]:
data["Depression"].value_counts()

0    29578
1     2838
Name: Depression, dtype: int64

In [57]:


# Replace the "Some College or AA Degree" category with "College"
data["Education Level"] = data["Education Level"].replace("Some College or AA Degree", "College or Degree")

data["Education Level"].value_counts()

College or Degree            9715
College Graduate or Above    7528
High School                  7517
9-11th Grade                 4541
Less Than 9th Grade          3115
Name: Education Level, dtype: int64

In [58]:
data["Marital Status"].value_counts()

Married          16781
Never Married     5765
Divorced          3619
Partner           2643
Widowed           2528
Separated         1080
Name: Marital Status, dtype: int64

In [59]:


# Define a custom function to categorize income
def categorize_income(income):
    if income in ["Below $5K", "Below $10K", "Below $15K", "Below $20K", "Below $25K", "Below $35K"]:
        return "Below 40k"
    else:
        return "Above 40k"

# Apply the custom function to the "Household Income" variable
data["Household Income"] = data["Household Income"].apply(categorize_income)

data["Household Income"].value_counts()

Above 40k    17916
Below 40k    14500
Name: Household Income, dtype: int64

In [60]:
data["Trouble Sleeping"] = data["Trouble Sleeping History"].replace({"Yes": 1, "No": 0})


In [61]:
data["Recreation"] = data["Vigorous Recreation"].replace({"Yes": 1, "No": 0})


In [62]:
data["Vigorous Work"] = data["Vigorous Work"].replace({"Yes": 1, "No": 0})

In [63]:
data["Moderate Work"] = data["Moderate Work"].replace({"Yes": 1, "No": 0})

In [64]:
data["Can Work"] = data["Cant Work"].replace({"Yes": 0, "No": 1})

In [65]:
data["Heart Problem"] = data["Health Problem Heart"].replace({"Yes": 1, "No": 0})
data["Diabetes"] = data["Health Problem Diabetes"].replace({"Yes": 1, "No": 0})

columns_to_drop = ["Trouble Sleeping History", "Vigorous Recreation", "Cant Work","Health Problem Diabetes","Health Problem Heart"]

# Use the drop() method to drop the columns
data = data.drop(columns_to_drop, axis=1)

In [66]:
data.head()

,Gender,Age,Education Level,Marital Status,Household Size,Household Income,Time In Current Job,Sleep Hours,Sedentary Time,Vigorous Work,Moderate Work,Depression,Trouble Sleeping,Recreation,Can Work,Heart Problem,Diabetes
0,0,44,College or Degree,Married,4,Above 40k,324.0,9.0,150.0,0,0,0,0,0,1,0,0
1,1,70,College Graduate or Above,Married,2,Above 40k,96.0,7.0,150.0,0,0,0,0,0,1,0,0
2,1,73,High School,Married,2,Below 40k,0.0,7.0,90.0,0,1,0,1,0,1,0,0
5,1,21,High School,Never Married,6,Below 40k,3.0,8.0,180.0,0,0,0,0,1,1,0,0
6,0,85,9-11th Grade,Widowed,1,Below 40k,0.0,8.0,300.0,0,0,0,0,0,1,0,0


In [67]:
data = pd.get_dummies(data, columns=["Education Level","Marital Status","Household Income"])

In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32416 entries, 0 to 36258
Data columns (total 27 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Gender                                     32416 non-null  int32  
 1   Age                                        32416 non-null  int64  
 2   Household Size                             32416 non-null  int64  
 3   Time In Current Job                        32416 non-null  float64
 4   Sleep Hours                                32416 non-null  float64
 5   Sedentary Time                             32416 non-null  float64
 6   Vigorous Work                              32416 non-null  int64  
 7   Moderate Work                              32416 non-null  int64  
 8   Depression                                 32416 non-null  int64  
 9   Trouble Sleeping                           32416 non-null  int64  
 10  Recreation            

In [69]:
data.head()

,Gender,Age,Household Size,Time In Current Job,Sleep Hours,Sedentary Time,Vigorous Work,Moderate Work,Depression,Trouble Sleeping,...,Education Level_High School,Education Level_Less Than 9th Grade,Marital Status_Divorced,Marital Status_Married,Marital Status_Never Married,Marital Status_Partner,Marital Status_Separated,Marital Status_Widowed,Household Income_Above 40k,Household Income_Below 40k
0,0,44,4,324.0,9.0,150.0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,1,70,2,96.0,7.0,150.0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,1,73,2,0.0,7.0,90.0,0,1,0,1,...,1,0,0,1,0,0,0,0,0,1
5,1,21,6,3.0,8.0,180.0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,1
6,0,85,1,0.0,8.0,300.0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [70]:
data.columns

Index(['Gender', 'Age', 'Household Size', 'Time In Current Job', 'Sleep Hours',
       'Sedentary Time', 'Vigorous Work', 'Moderate Work', 'Depression',
       'Trouble Sleeping', 'Recreation', 'Can Work', 'Heart Problem',
       'Diabetes', 'Education Level_9-11th Grade',
       'Education Level_College Graduate or Above',
       'Education Level_College or Degree', 'Education Level_High School',
       'Education Level_Less Than 9th Grade', 'Marital Status_Divorced',
       'Marital Status_Married', 'Marital Status_Never Married',
       'Marital Status_Partner', 'Marital Status_Separated',
       'Marital Status_Widowed', 'Household Income_Above 40k',
       'Household Income_Below 40k'],
      dtype='object')

In [71]:
# Divide the data into independent and target variables
X = data.drop("Depression", axis=1) # Independent variables
y = data["Depression"] # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [72]:

# Train a logistic regression model
svm = SVC()
svm.fit(X_train, y_train)

# Evaluate the performance of the model on the testing set
score = svm.score(X_test, y_test)
print("Accuracy on testing set: {:.2f}%".format(score * 100))

Accuracy on testing set: 91.38%


In [73]:
import json

# Example variable to export
send = {'accuracy':(score * 100)}

with open('accuracyv.json', 'w') as f:
    json.dump(send, f)

In [74]:
X_test.head()

,Gender,Age,Household Size,Time In Current Job,Sleep Hours,Sedentary Time,Vigorous Work,Moderate Work,Trouble Sleeping,Recreation,...,Education Level_High School,Education Level_Less Than 9th Grade,Marital Status_Divorced,Marital Status_Married,Marital Status_Never Married,Marital Status_Partner,Marital Status_Separated,Marital Status_Widowed,Household Income_Above 40k,Household Income_Below 40k
22722,1,59,4,84.0,7.0,660.0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1267,1,54,4,432.0,6.0,30.0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0
5436,1,75,2,0.0,9.0,60.0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
13793,1,66,3,180.0,7.0,720.0,0,1,0,1,...,0,0,0,1,0,0,0,0,1,0
16814,0,48,3,0.0,7.0,300.0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [75]:
import pickle
with open('sv_model.pickle', 'wb') as f:
    pickle.dump(svm, f)

print("complete")

complete
